In [ ]:
!pip install python-chess
!apt-get install stockfish

In [ ]:
!pip install numpy

In [1]:
import chess
import chess.engine
import random
import numpy as np

In [2]:
def random_board(max_depth=200):
    # Randomly determine the stage of the game
    if random.random() < 0.2:
        max_depth = min(15, max_depth)  # Consider shallower depths for more opening positions
    elif random.random() < 0.5:
        max_depth = min(50, max_depth)  # Consider medium depths for mid-game positions

    board = chess.Board()
    depth = random.randrange(0, max_depth)

    for _ in range(depth):
        all_moves = list(board.legal_moves)
        if not all_moves:  # Break if no legal moves are available
            break

        random_move = random.choice(all_moves)
        board.push(random_move)

        if board.is_game_over():
            break

    return board


# creating dataset

In [21]:
squares_index= {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


In [20]:
def split_dims(board):
  board3d=np.zeros((14,8,8),dtype=np.int8) #3d representation of board

  # add pieces to board representation
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece,chess.WHITE):
      idx=np.unravel_index(square,(8,8))
      board3d[piece-1][7-idx[0]][idx[1]]=1

    for square in board.pieces(piece,chess.BLACK):
      idx=np.unravel_index(square,(8,8))
      board3d[piece+5][7- idx[0]][idx[1]]=1

  #adding valid moves
  aux=board.turn
  board.turn=chess.WHITE
  for move in board.legal_moves:
    i,j=square_to_index(move.to_square)
    board3d[12][i][j]=1
  board.turn=chess.BLACK
  for move in board.legal_moves:
    i,j=square_to_index(move.to_square)
    board3d[13][i][j]=1
  board.turn=aux

  return board3d

In [ ]:
!pip install tensorflow-gpu==2.10.0

In [4]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
!cp /content/drive/MyDrive/dataset/dataset.npz  /content/


In [ ]:
# Load the previous dataset if it exists
try:
    previous_data = np.load('dataset.npz', allow_pickle=True)
    previous_inputs = previous_data['inputs']
    previous_labels = previous_data['labels']
except FileNotFoundError:
    previous_inputs = np.array([])
    previous_labels = np.array([])

# Generate new data
num_samples = 100000
new_input_data_list = []
new_labels_list = []

for i in range(num_samples):
    board = random_board()
    x = split_dims(board)
    y = stockfish(board, 10)
    if y is None:
        i -= 1
        continue
    new_input_data_list.append(x)
    new_labels_list.append(y)

# Convert lists to numpy arrays
new_input_data_array = np.array(new_input_data_list)
new_labels_array = np.array(new_labels_list)

# Concatenate new data with previous data
combined_inputs = np.concatenate([previous_inputs, new_input_data_array], axis=0)
combined_labels = np.concatenate([previous_labels, new_labels_array], axis=0)

# Convert NumPy arrays to TensorFlow tensors
combined_inputs_tensor = tf.convert_to_tensor(combined_inputs, dtype=tf.float32)
combined_labels_tensor = tf.convert_to_tensor(combined_labels, dtype=tf.float32)

# Save the combined dataset to an NPZ file
np.savez('dataset.npz', inputs=combined_inputs_tensor.numpy(), labels=combined_labels_tensor.numpy())


In [ ]:
!cp /content/dataset.npz /content/drive/MyDrive/dataset/


# Model

In [5]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


In [6]:
from keras.activations import selu
from keras import regularizers


def build_model(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))

    x = board3d
    for _ in range(conv_depth):
        x = layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same', activation='relu', data_format='channels_first')(x)
        x = layers.Dropout(0.25)(x)  # Apply dropout regularization in convolutional layers

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)  # Apply Elastic Net regularization in dense layers
    x = layers.Dropout(0.5)(x)  # Apply dropout regularization in dense layers
    x = layers.Dense(1, activation='selu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)  # Apply Elastic Net regularization in dense layers
    #x = layers.Lambda(lambda x: x * 10)(x)
    return models.Model(inputs=board3d, outputs=x)



In [7]:
from keras.activations import selu
from keras import regularizers


def build_ResNet_model(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))

    # Convolutional layers
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(board3d)

    for _ in range(conv_depth):
        previous = x
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Add()([x, previous])

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1, activation=selu)(x)
    return models.Model(inputs=board3d, outputs=x)


# Importing Data

In [9]:
import tensorflow.keras.callbacks as callbacks

In [11]:

def get_dataset():
  container=np.load('D:\Repos\Chess Engine\dataset.npz', allow_pickle=True)
  b,v=container['inputs'],container['labels']
  v=np.asarray(v/1e4,dtype=np.float32)
  return b,v

x_train,y_train=get_dataset()

In [13]:
print(x_train.shape)

(282977, 14, 8, 8)


In [14]:
#model=build_model(32,4)
model=build_ResNet_model(32,4)

In [15]:
from keras.losses import Huber

model.compile(optimizer='adam',loss=Huber())
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 14, 8, 8)]   0           []                               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 32, 8, 8)     4064        ['input_2[0][0]']                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 32, 8, 8)     9248        ['conv2d_9[0][0]']               
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 32, 8, 8)    32          ['conv2d_10[0][0]']              
 rmalization)                                                                               

In [17]:
model.fit(x=x_train,y=y_train,
          batch_size=512,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='val_loss',patience=10),callbacks.EarlyStopping(monitor='val_loss',patience=15,min_delta=1e-5)]
          )

model.save('model.h5')

Epoch 1/1000
498/498 [==============================] - 12s 25ms/step - loss: 0.0053 - val_loss: 0.0052 - lr: 0.0010
Epoch 2/1000
498/498 [==============================] - 12s 25ms/step - loss: 0.0046 - val_loss: 0.0038 - lr: 0.0010
Epoch 3/1000
498/498 [==============================] - 12s 25ms/step - loss: 0.0042 - val_loss: 0.0104 - lr: 0.0010
Epoch 4/1000
498/498 [==============================] - 13s 25ms/step - loss: 0.0040 - val_loss: 0.0036 - lr: 0.0010
Epoch 5/1000
498/498 [==============================] - 13s 25ms/step - loss: 0.0039 - val_loss: 0.0036 - lr: 0.0010
Epoch 6/1000
498/498 [==============================] - 13s 25ms/step - loss: 0.0038 - val_loss: 0.0033 - lr: 0.0010
Epoch 7/1000
498/498 [==============================] - 13s 25ms/step - loss: 0.0036 - val_loss: 0.0032 - lr: 0.0010
Epoch 8/1000
498/498 [==============================] - 13s 25ms/step - loss: 0.0036 - val_loss: 0.0056 - lr: 0.0010
Epoch 9/1000
498/498 [==============================] - 13s 26ms

In [ ]:
!cp /content/model.h5 /content/drive/MyDrive/dataset/


# Testing

In [14]:
model = tf.keras.models.load_model("D:\Repos\Chess Engine\model.h5", compile=False)


In [15]:
board=random_board()
board3d=split_dims(board)
board3d=np.expand_dims(board3d,0)
print(model.predict(board3d)[0][0]*1e4)


1/1 [==============================] - 6s 6s/step
-165.4353365302086


In [24]:
temp=split_dims(random_board())
temp.shape

(14, 8, 8)

In [33]:
num_samples = 100
input_data_list = []
labels_list = []
boards=[]
temp=[]
for i in range(num_samples):
    board = random_board()
    boards.append(board)
for board in boards:
    x = split_dims(board)
    temp.append(x)

x_input=np.array(temp)
x_input=tf.convert_to_tensor(x_input, dtype=tf.float32)
pred =model.predict(x_input)
max_index = np.argmax(pred)
temp=boards[max_index]
#print(f'Prediction: {pred*1e4}')


4/4 [==============================] - 0s 8ms/step
